In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.datasets import  fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchvision import transforms
from tqdm import tqdm

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Preparing the data

In [6]:
data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train.shape

(16512, 8)

In [8]:
X_test.shape

(4128, 8)

In [9]:
toTensorTransform = transforms.Lambda(lambda x: torch.tensor(x, dtype=torch.float32))

In [15]:
class A:
    def __len__(self):
        return 3
    
    def __getitem__(self, index):
        return index

a = A()
# len(a)
a[6]

6

In [17]:
# Create a custom PyTorch dataset
class CaliforniaHousingDataset(Dataset):
    def __init__(self, features, targets, transform=None, target_transform=None):
        self.features = features
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.targets[index]
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y

In [19]:
train_dataset = CaliforniaHousingDataset(X_train, y_train, transform=toTensorTransform, target_transform=toTensorTransform)
len(train_dataset)
train_dataset[5]

(tensor([ 1.4399, -0.6831,  0.3548, -0.2030,  1.1157,  0.0231,  0.8316, -1.1357]),
 tensor(2.6480))

In [20]:
# Create instances of the custom dataset
train_dataset = CaliforniaHousingDataset(X_train, y_train, transform=toTensorTransform, target_transform=toTensorTransform)
test_dataset = CaliforniaHousingDataset(X_test, y_test, transform=toTensorTransform, target_transform=toTensorTransform)

# Create PyTorch data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

## Building your model

In [21]:
class MyLinear(nn.Module):

    def __init__(self, in_f, out_f, bias=True, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bias = bias
        self.w = nn.parameter.Parameter(torch.empty(out_f, in_f))
        if bias:
            self.b = nn.parameter.Parameter(torch.empty(out_f)) 
        self.reset_params()   ### initialization paramters
    
    def reset_params(self):
        nn.init.kaiming_normal_(self.w, nonlinearity="relu")
        if self.bias:
            nn.init.constant_(self.b, 0)

    def forward(self, x):
        return F.linear(x, self.w, self.b)

In [37]:
def build_model(input_size, hidden_size, output_size):
    model = nn.Sequential(nn.Linear(input_size, hidden_size), 
                           nn.ReLU(),
                           nn.Linear(hidden_size, hidden_size), 
                           nn.ReLU(),  
                           nn.Linear(hidden_size, output_size))
    return model

# Train & Test

In [23]:
### custom LOSS function

class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta

    def forward(self, predicted, target):
        residual = torch.abs(predicted - target)
        loss = torch.where(residual < self.delta, 0.5 * residual**2, self.delta * (residual - 0.5 * self.delta))
        return torch.mean(loss)

In [38]:
num_epochs = 100

model = build_model(8, 300, 1)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [39]:
for epoch in tqdm(range(num_epochs)):
    
    losses = []
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        targer = target.to(device)

        output = model(data)

        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    if epoch % 20 == 19:
        print(f"Loss average per epoch {epoch + 1} is {sum(losses)/len(losses):.4f}")
        


  0%|          | 0/100 [00:00<?, ?it/s]/home/ashkan/Documents/deeplearning_bootcamp_pytorch/venv/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
 20%|██        | 20/100 [00:53<03:33,  2.67s/it]

Loss average per epoch 20 is 1.3389


 40%|████      | 40/100 [02:27<04:49,  4.82s/it]

Loss average per epoch 40 is 1.3372


 60%|██████    | 60/100 [04:04<03:09,  4.74s/it]

Loss average per epoch 60 is 1.3380


 80%|████████  | 80/100 [05:50<01:04,  3.21s/it]

Loss average per epoch 80 is 1.3376


100%|██████████| 100/100 [07:01<00:00,  4.22s/it]

Loss average per epoch 100 is 1.3372


In [40]:
with torch.no_grad():
    for data, target in test_loader:
        y_pred = model(data[:10])
        print("y_true", target[:10])
        print("y_pred", y_pred)
        break

y_true tensor([0.4770, 0.4580, 5.0000, 2.1860, 2.7800, 1.5870, 1.9820, 1.5750, 3.4000,
        4.4660])
y_pred tensor([[2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836],
        [2.0836]])


In [ ]:
k = 4
num_val_samples = len(train_dataset) // k
num_epochs = 100